In [1]:
## Python Core
import datetime
import re
import math
from IPython.display import Markdown as md

## Data manipulation
import pandas as pd
import numpy as np


## Data Vizualization
from pprint import pprint
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib as mpl




In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:,.2f}'.format)
pd.set_option('display.max_colwidth', None)

In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\(.*\)", "", text)
    text = re.sub(r"[^a-z]", " ", text)
    text = text.strip()
    text = re.sub(r"\s+", "_", text)
    return text

# Select Dates

In [ ]:
quarters = {
    2014: range(1, 5),
    # 2015: range(1, 5),
    # 2016: range(1, 5),
    # 2017: range(1, 5),
    # 2018: range(1, 5),
    # 2019: range(1, 3),
}

# Reading loan origination data

In [4]:
loan_characteristic_columns_original_format = """Credit Score
First Payment Date
First Time Homebuyer Flag
Maturity Date
Metropolitan Statistical Area (MSA) Or Metropolitan Division
Mortgage Insurance Percentage (MI %)
Number of Units
Occupancy Status
Original Combined Loan-to-Value (CLTV)
Original Debt-to-Income (DTI) Ratio
Original UPB
Original Loan-to-Value (LTV)
Original Interest Rate
Channel
Prepayment Penalty Mortgage (PPM) Flag
Amortization Type (Formerly Product Type)
Property State
Property Type
Postal Code
Loan Sequence Number
Loan Purpose
Original Loan Term
Number of Borrowers
Seller Name
Servicer Name
Super Conforming Flag
Pre-HARP Loan Sequence Number
Program Indicator
HARP Indicator
Property Valuation Method
Interest Only (I/O) Indicator"""
loan_characteristic_columns_original_format = loan_characteristic_columns_original_format.split("\n")
loan_characteristic_columns = [clean_text(loan_char_column) for loan_char_column in loan_characteristic_columns_original_format]

## Selecting columns to read

In [5]:
selected_loan_characteristic_columns = ['credit_score', 
                                        'metropolitan_statistical_area_or_metropolitan_division', 
                                        'original_combined_loan_to_value', 
                                        'original_debt_to_income_ratio', 
                                        'original_loan_to_value', 
                                        'property_state', 
                                        'postal_code', 
                                        'loan_sequence_number']
selected_loan_characteristic_column_indices = [loan_characteristic_columns.index(selected_column) for selected_column in selected_loan_characteristic_columns]
print(selected_loan_characteristic_column_indices)
print(selected_loan_characteristic_columns)

[0, 4, 8, 9, 11, 16, 18, 19]
['credit_score', 'metropolitan_statistical_area_or_metropolitan_division', 'original_combined_loan_to_value', 'original_debt_to_income_ratio', 'original_loan_to_value', 'property_state', 'postal_code', 'loan_sequence_number']


In [6]:
loan_origination_files = [f"./data/historical_data_{year}Q{q}.txt" for year in quarters for q in quarters[year]]
print("Origination Files")
pprint(loan_origination_files)

Origination Files
['./data/historical_data_2014Q1.txt',
 './data/historical_data_2014Q2.txt',
 './data/historical_data_2014Q3.txt',
 './data/historical_data_2014Q4.txt']


## Read origination CSV 

In [7]:
loan_origination_df = []
for loan_origin_filename in loan_origination_files:
    loan_origination_quarter_df = pd.read_csv(loan_origin_filename, 
                                              delimiter="|", 
                                              usecols=selected_loan_characteristic_column_indices, 
                                              header=None)
    loan_origination_df.append(loan_origination_quarter_df)
    
loan_origination_df = pd.concat(loan_origination_df, ignore_index=True)
loan_origination_df.columns = selected_loan_characteristic_columns

loan_origination_df["origination_quarter"] = [loan_sequence_number[1:5] for loan_sequence_number in loan_origination_df["loan_sequence_number"].to_numpy()]

In [15]:
print(loan_origination_df.shape)
loan_origination_df.head()

(1142388, 9)


,credit_score,metropolitan_statistical_area_or_metropolitan_division,original_combined_loan_to_value,original_debt_to_income_ratio,original_loan_to_value,property_state,postal_code,loan_sequence_number,origination_quarter
0,629,"17,300.00",77,45,71,KY,42200,F14Q10000001,14Q1
1,770,NaN,89,30,89,NY,13600,F14Q10000002,14Q1
2,674,NaN,89,999,76,MI,49200,F14Q10000003,14Q1
3,717,"39,300.00",77,41,77,RI,2800,F14Q10000004,14Q1
4,813,"19,780.00",95,32,95,IA,50300,F14Q10000005,14Q1


# Read loan performance data

In [9]:
loan_perf_columns_original_format = """Loan Sequence Number
Monthly Reporting Period
Current Actual UPB
Current Loan Delinquency Status
Loan Age
Remaining Months to Legal Maturity
Defect Settlement Date
Modification Flag
Zero Balance Code
Zero Balance Effective Date
Current Interest Rate
Current Deferred UPB
Due Date of Last Paid Installment (DDLPI)
MI Recoveries
Net Sales Proceeds
Non MI Recoveries
Expenses
Legal Costs
Maintenance and Preservation Costs
Taxes and Insurance
Miscellaneous Expenses
Actual Loss Calculation
Modification Cost
Step Modification Flag
Deferred Payment Plan
Estimated Loan-to-Value (ELTV)
Zero Balance Removal UPB
Delinquent Accrued Interest
Delinquency Due to Disaster
Borrower Assistance Status Code
Current Month Modification Cost
Interest Bearing UPB"""
loan_perf_columns_original_format = loan_perf_columns_original_format.split("\n")
loan_performance_columns = [clean_text(loan_performance_column) for loan_performance_column in loan_perf_columns_original_format]

## Selecting columns to read

In [10]:
selected_loan_performance_columns = ['loan_sequence_number',
                                     'monthly_reporting_period',
                                     'current_actual_upb',
                                     'current_loan_delinquency_status',
                                     'loan_age',
                                     'remaining_months_to_legal_maturity',
                                     'zero_balance_effective_date',
                                     'net_sales_proceeds',
                                     'actual_loss_calculation',
                                     'estimated_loan_to_value',
                                     'zero_balance_removal_upb',
                                     'delinquency_due_to_disaster']

selected_loan_performance_column_indices = [loan_performance_columns.index(selected_column) for selected_column in selected_loan_performance_columns]
print(selected_loan_performance_column_indices)
pprint(selected_loan_performance_columns)

[0, 1, 2, 3, 4, 5, 9, 14, 21, 25, 26, 28]
['loan_sequence_number',
 'monthly_reporting_period',
 'current_actual_upb',
 'current_loan_delinquency_status',
 'loan_age',
 'remaining_months_to_legal_maturity',
 'zero_balance_effective_date',
 'net_sales_proceeds',
 'actual_loss_calculation',
 'estimated_loan_to_value',
 'zero_balance_removal_upb',
 'delinquency_due_to_disaster']


In [11]:
loan_performance_files = [f"./data/historical_data_time_{year}Q{q}.txt" for year in quarters for q in quarters[year]]

print("Performance Files")
pprint(loan_performance_files)

Performance Files
['./data/historical_data_time_2014Q1.txt',
 './data/historical_data_time_2014Q2.txt',
 './data/historical_data_time_2014Q3.txt',
 './data/historical_data_time_2014Q4.txt']


## Read performance CSV

In [12]:
loan_performance_df = []
for loan_performance_filename in loan_performance_files:
    loan_performance_quarter_df = pd.read_csv(loan_performance_filename, 
                                              delimiter="|", 
                                              usecols=selected_loan_performance_column_indices,
                                              header=None)
    loan_performance_df.append(loan_performance_quarter_df)
    
loan_performance_df = pd.concat(loan_performance_df, ignore_index=True)
loan_performance_df.columns = selected_loan_performance_columns

C:\Users\ashwi\anaconda3\envs\Credit_Risk_Modeling\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (3,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ashwi\anaconda3\envs\Credit_Risk_Modeling\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
print(loan_performance_df.shape)
loan_performance_df.head()

(66798436, 12)


,loan_sequence_number,monthly_reporting_period,current_actual_upb,current_loan_delinquency_status,loan_age,remaining_months_to_legal_maturity,zero_balance_effective_date,net_sales_proceeds,actual_loss_calculation,estimated_loan_to_value,zero_balance_removal_upb,delinquency_due_to_disaster
0,F14Q10000001,201404,"324,000.00",0,0,180,NaN,NaN,NaN,NaN,NaN,NaN
1,F14Q10000001,201405,"322,000.00",0,1,179,NaN,NaN,NaN,NaN,NaN,NaN
2,F14Q10000001,201406,"321,000.00",0,2,178,NaN,NaN,NaN,NaN,NaN,NaN
3,F14Q10000001,201407,"320,000.00",0,3,177,NaN,NaN,NaN,NaN,NaN,NaN
4,F14Q10000001,201408,"320,000.00",0,4,176,NaN,NaN,NaN,NaN,NaN,NaN
